In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("check").getOrCreate()
print(spark.version)


ModuleNotFoundError: No module named 'pyspark'

In [3]:
from pyspark.sql import SparkSession

# Create SparkSession with S3A configs
spark = (
    SparkSession.builder
    .appName("s3-verify")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")  # useful for MinIO/localstack; works fine on AWS too
    .getOrCreate()
)

print("✅ Spark Session started")

# 🔍 Replace with your bucket + file
bucket = "my-bucket-name"
key = "sample-data/test.csv"
s3_path = f"s3a://{bucket}/{key}"

# Try reading CSV
try:
    df = spark.read.csv(s3_path, header=True, inferSchema=True)
    print(f"✅ Successfully read file from {s3_path}")
    df.show(5)
except Exception as e:
    print("❌ Failed to read from S3:", str(e))

# Optionally: List files in bucket prefix
try:
    files = spark._jsc.hadoopConfiguration().get("fs.s3a.bucket")
    print("S3 path test:", s3_path)
    print("Listing files:")
    spark._jsc.hadoopConfiguration().iterator()
except Exception as e:
    print("⚠️ Listing files not supported in this way:", str(e))


✅ Spark Session started
❌ Failed to read from S3: An error occurred while calling o35.csv.
: java.lang.NoClassDefFoundError: com/amazonaws/auth/AWSCredentialsProvider
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:467)
	at org.apache.hadoop.conf.Configuration.getClassByNameOrNull(Configuration.java:2625)
	at org.apache.hadoop.conf.Configuration.getClassByName(Configuration.java:2590)
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2686)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T

spark = SparkSession.builder.appName("check").getOrCreate() # type: ignore # ignore

df: DataFrame = (
    spark.read.format("csv")
    .option("inferSchema", True)
    .option("header", True)
    .load("fk.csv")
)
df.show()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [10]:
try: spark.stop()
except: pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("s3-read")
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    # optional if bucket is in ap-south-1
    .config("spark.hadoop.fs.s3a.endpoint", "s3.ap-south-1.amazonaws.com")
    .getOrCreate()
)


In [1]:
# clean start
try: spark.stop()
except: pass

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("s3-check").getOrCreate()

# jars present?
spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
print("S3A present ✅")

# (optional) confirm which credentials provider Spark will use
print("Creds provider:",
      spark.sparkContext._jsc.hadoopConfiguration().get("fs.s3a.aws.credentials.provider"))


Py4JJavaError: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:3

In [9]:
# 1) Did Spark pick up the packages flag?
print("packages:", spark.sparkContext.getConf().get("spark.jars.packages", None))

# 2) Is the S3A class actually present?
try:
    spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
    print("S3A jar present ✅")
except Exception as e:
    print("S3A jar missing ❌", e)

packages: org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262
S3A jar missing ❌ An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(

In [13]:
import os
# Add the S3 connector jars via packages
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.4,"
    "com.amazonaws:aws-java-sdk-bundle:1.12.262 pyspark-shell"
)

# (Re)start Spark cleanly
try: spark.stop()
except: pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("s3-read")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    # Helpful for ap-south-1
    .config("spark.hadoop.fs.s3a.endpoint", "s3.ap-south-1.amazonaws.com")
    .getOrCreate()
)

# Verify jars are present
print("packages:", spark.sparkContext.getConf().get("spark.jars.packages"))
spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
print("S3A ready ✅")


packages: None


Py4JJavaError: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:3

In [1]:
# Clean start
try: spark.stop()
except: pass

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("s3-read")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", "s3.ap-south-1.amazonaws.com")  # your region
    .getOrCreate()
)

# verify the jar is actually loaded
spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
print("S3A present ✅")


Py4JJavaError: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:3

In [3]:
import boto3; print(boto3.client("sts").get_caller_identity())

{'UserId': 'AROAZI2LGOPW3CVN6PGSU:i-0399e8193fe8f80bc', 'Account': '637423481837', 'Arn': 'arn:aws:sts::637423481837:assumed-role/server/i-0399e8193fe8f80bc', 'ResponseMetadata': {'RequestId': 'a73d1118-f73b-43aa-a30b-7e5f993fc3ae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a73d1118-f73b-43aa-a30b-7e5f993fc3ae', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6MTc1NTg1OTI2MTI3NDpHOnU0N0t3NjNp', 'content-type': 'text/xml', 'content-length': '451', 'date': 'Fri, 22 Aug 2025 10:41:01 GMT'}, 'RetryAttempts': 0}}
